In [307]:
import pandas as pd
import numpy as np
import random
import math
import re
from scipy.special import psi #digamma function
from scipy.special import polygamma #derivative of digamma function
from scipy.special import loggamma
from IPython.display import display
import numpy.random
np.random.seed(1)

In [308]:
def Generate_Data(num_topic,voc_size,num_doc,xi,alpha,eta):
    num_word=np.random.poisson(xi,num_doc) #number of words in each document
    
    beta=np.zeros([num_topic,voc_size])
    for i in range(num_topic):
        beta[i]=np.random.dirichlet(eta)
    
    theta=np.zeros([num_doc,num_topic])
    for d in range(num_doc):
        theta[d]=np.random.dirichlet(alpha)
    
    doc={}
    for d in range(num_doc):
        W=np.zeros([V,num_word[d]])
        for n in range(num_word[d]):
            z=np.random.multinomial(1,theta[d])
            topic_index=np.where(z==1)[0][0]
            w=np.random.multinomial(1,beta[topic_index,])
            W[:,n]=w
        doc[d]=W
                            
    return doc,beta,theta  

In [309]:
class document:
    def __init__(self,words,words_num,phi,gamma):
        self.words=words #words matrix
        self.words_num=words_num #number of words
        self.phi=phi 
        self.gamma=gamma

In [310]:
def sim_data_matrix(doc,V):
    M=int(len(doc))
    N=np.zeros(M,dtype=np.int)   
    doc_matrix=[]
    for d in range(M):
        N[d]=int(doc[d].shape[1]) #number of words in each document
        gamma=np.zeros(K)
        phi=np.zeros([N[d],K])
        
        doc_matrix.append(document(doc[d],N[d],phi,gamma))
    return doc_matrix,N

In [311]:
def Updatealpha():
    global alpha,alpha_old

    for i in range(0,K):
        alpha_old[i]=alpha[i]
        
    g=np.zeros(K)
    h=np.zeros(K)
    z=0
    c=0
    for i in range(0,K):
        h[i]=-M*polygamma(1,alpha_old[i])
        g[i]=M*(psi(sum(alpha_old))-psi(alpha_old[i]))
        for d in range(0,M):
            g[i]+=(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
    z=M*polygamma(1,sum(alpha_old))
    sum_gh=0
    sum_h=0
    for i in range(0,K):
        sum_gh+=(g[i]/h[i])
        sum_h+=(1/h[i])
    c=sum_gh/(1/z+sum_h)        
    
    for i in range(0,K):
        alpha[i]=alpha_old[i]-((g[i]-c)/h[i])

In [312]:
def Updatebeta():
    for i in range(0,K):
        sum_beta=0
        for j in range(0,V):
            beta[i][j]=0
            for d in range(0,M):
                for n in range(0,N[d]):
                    beta[i][j]+=doc_matrix[d].phi[n][i]*doc_matrix[d].words[j][n]
            sum_beta+=beta[i][j]
        
        #normalize beta
        for j in range(0,V):
            beta[i][j]=beta[i][j]/sum_beta
    print("Update of beta DONE")

In [313]:
def Initial_gamma_phi():
    for d in range(M):
        for i in range(K):
            doc_matrix[d].gamma[i]=alpha[i]+N[d]/K
            for n in range(N[d]):
                doc_matrix[d].phi[n][i]=1.0/K

In [314]:
def Updategamma(d):
        
    for i in range(0,K):
        gamma_old[d][i]=doc_matrix[d].gamma[i]
        sum_of_phi=0
        for n in range(0,N[d]):
            sum_of_phi+=doc_matrix[d].phi[n][i]

        doc_matrix[d].gamma[i]=alpha[i]+sum_of_phi    

In [315]:
def Updatephi(d):

    for n in range(0,N[d]):
        phi_old[d][n]=doc_matrix[d].phi[n]
        phi_sum=0
        for i in range(0,K):
            j=np.where(np.array(doc_matrix[d].words[:,n]==1))#find out the n-th word index
            doc_matrix[d].phi[n][i]=beta[i][j]*math.exp(psi(doc_matrix[d].gamma[i]))
            phi_sum += doc_matrix[d].phi[n][i]
        
        #normalize phi[d][n]
        for i in range(0,K):
            doc_matrix[d].phi[n][i]=doc_matrix[d].phi[n][i]/phi_sum

In [316]:
def VariationalInference():
    
    print("E-STEP: Variational Inference START")
    judge=0
    for d in range(M):
        if ((abs(doc_matrix[d].gamma-gamma_old[d])>error).any() or (abs(doc_matrix[d].phi-phi_old[d])>error).any()):
            judge=1
            
    t=0        
    while (judge==1):
        t+=1
        print("This is iteration",t,"for updating variational parameters")
        
        for d in range(M):
            Updatephi(d)
            Updategamma(d)
        judge=0
        for d in range(M):
            if ((abs(doc_matrix[d].gamma-gamma_old[d])>error).any() or (abs(doc_matrix[d].phi-phi_old[d])>error).any()):
                judge=1
    print("E-STEP: Variational Inference END")

In [317]:
def M_step():
    print("M-STEP: Model parameters update START")
    
    t=1
    print("This is iteration",t,"for updating alpha")
    Updatealpha()
    print(alpha)
    while((abs(alpha-alpha_old)>error).any()):
        t+=1
        print("This is iteration",t,"for updating alpha")
        Updatealpha()
        print(alpha)
    
    Updatebeta()
    print("M-STEP: Model parameters update END")

In [318]:
def UpdateL():
    l=0
    for d in range(0,M):
        l+=loggamma(sum(alpha))-loggamma(sum(doc_matrix[d].gamma))
        for i in range(0,K):
            l+=-loggamma(alpha[i])+(alpha[i]-1)*(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
            l+=loggamma(doc_matrix[d].gamma[i])-(doc_matrix[d].gamma[i]-1)*(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
            for n in range(0,N[d]):
                l+=doc_matrix[d].phi[n][i]*(psi(doc_matrix[d].gamma[i])-psi(sum(doc_matrix[d].gamma)))
                l+=-doc_matrix[d].phi[n][i]*math.log(doc_matrix[d].phi[n][i])
                for j in range(0,V):
                    if (beta[i][j]>0):
                        l+=doc_matrix[d].phi[n][i]*doc_matrix[d].words[j][n]*math.log(beta[i][j])
                    
    return l
    

In [352]:
np.random.seed(1)
K=2 #number of topics
V=10 #Vocabulary size
m=50 #number of document
xi=7
true_alpha=np.array([1.5,0.5])
#true_alpha=[0.5 for i in range(K)]
true_eta=[0.9 for i in range(V)]
doc,true_beta,true_theta=Generate_Data(K,V,m,xi,true_alpha,true_eta)#simulated data


doc_matrix,N=sim_data_matrix(doc,V)

M=len(doc_matrix) #number of documents
error=0.01
L=0 #lower bound of loglikelihood

gamma_old={}
phi_old={}
alpha_old=np.zeros(K)
for d in range(M):
    gamma_old[d]=np.zeros(K)
    phi_old[d]=np.zeros([N[d],K])

In [353]:
#initialize alpha and beta

alpha=np.array([1.0,2.0])

beta=np.zeros([K,V])
for i in range(0,K):
    for j in range(0,V):
        beta[i][j]=1/V



In [354]:
iteration=1
print("------This is iteration",iteration,"------")
Initial_gamma_phi()

for d in range(M):
    Updatephi(d)
    Updategamma(d)
VariationalInference()
M_step()

new_L=UpdateL()
print("Lower bound of the log likelihood is", new_L)
while(abs(L-new_L)>error):
    L=new_L
    iteration+=1
    print("------This is iteration",iteration,"------")
    Initial_gamma_phi()
    for d in range(M):
        Updatephi(d)
        Updategamma(d)
    VariationalInference()
    M_step()    
    new_L=UpdateL()
    print("Lower bound of the log likelihood is", new_L)    


------This is iteration 1 ------
E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
This is iteration 4 for updating variational parameters
This is iteration 5 for updating variational parameters
This is iteration 6 for updating variational parameters
This is iteration 7 for updating variational parameters
This is iteration 8 for updating variational parameters
This is iteration 9 for updating variational parameters
This is iteration 10 for updating variational parameters
This is iteration 11 for updating variational parameters
This is iteration 12 for updating variational parameters
This is iteration 13 for updating variational parameters
This is iteration 14 for updating variational parameters
This is iteration 15 for updating variational parameters
This is iteration 16 for updating variational parameters
This is iteration 17 for upd

This is iteration 3 for updating variational parameters
This is iteration 4 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[15.55493374 44.24438017]
This is iteration 2 for updating alpha
[15.74104625 44.78511665]
This is iteration 3 for updating alpha
[15.74335037 44.79181223]
Update of beta DONE
M-STEP: Model parameters update END
Lower bound of the log likelihood is -749.6481646612839
------This is iteration 10 ------
E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
This is iteration 4 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[17.28091433 49.25842887]
This is iteration 2 for updating alpha
[17.44967524 49.74875634]
This is itera

This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[ 38.03595281 109.55526166]
This is iteration 2 for updating alpha
[ 38.1149181  109.78467961]
This is iteration 3 for updating alpha
[ 38.11508388 109.78516124]
Update of beta DONE
M-STEP: Model parameters update END
Lower bound of the log likelihood is -747.9981661865322
------This is iteration 23 ------
E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[ 39.76900395 114.59017704]
This is iteration 2 for updating alpha
[ 39.84459187 114.80978293]
Thi

E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[ 62.32966895 180.13492847]
This is iteration 2 for updating alpha
[ 62.37819388 180.27590853]
This is iteration 3 for updating alpha
[ 62.3782319  180.27601898]
Update of beta DONE
M-STEP: Model parameters update END
Lower bound of the log likelihood is -747.5497202568362
------This is iteration 37 ------
E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
This is iteration 3 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[ 64.06695077 185.18222605]
This is iteration 2 for updating a

Lower bound of the log likelihood is -747.3525260060467
------This is iteration 51 ------
E-STEP: Variational Inference START
This is iteration 1 for updating variational parameters
This is iteration 2 for updating variational parameters
E-STEP: Variational Inference END
M-STEP: Model parameters update START
This is iteration 1 for updating alpha
[ 88.40757782 255.89678974]
This is iteration 2 for updating alpha
[ 88.44188539 255.99646073]
This is iteration 3 for updating alpha
[ 88.44189876 255.99649957]
Update of beta DONE
M-STEP: Model parameters update END
Lower bound of the log likelihood is -747.3425908934564


In [355]:

df1=pd.DataFrame(beta)

In [356]:

df0=pd.DataFrame(true_beta)

In [357]:
display(alpha)

array([ 88.44189876, 255.99649957])

In [358]:
display(df0)

,0,1,2,3,4,5,6,7,8,9
0,0.129833,0.001595,0.152844,0.110542,0.058465,0.132254,0.097387,0.255110,0.021326,0.040644
1,0.122110,0.147718,0.046655,0.064109,0.119466,0.006704,0.177087,0.172659,0.068915,0.074578


In [359]:
display(df1)

,0,1,2,3,4,5,6,7,8,9
0,0.140960,0.073956,0.100007,0.102501,0.073346,0.099214,0.087764,0.228466,0.041061,0.052725
1,0.140695,0.073093,0.099603,0.102687,0.073303,0.099876,0.088050,0.228833,0.041054,0.052807
